In [1]:
from __future__ import print_function

import os
from PIL import Image
from skimage.io import imsave
from random import shuffle
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K

from data import *
from keras.preprocessing.image_augmentation import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
img_rows = 256
img_cols = 256

smooth = 1.


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def get_unet():
    inputs = Input((img_rows, img_cols, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model

def load_data():
    mydata = dataProcess(img_rows, img_cols)
    imgs_train, imgs_mask_train = mydata.load_train_data()
    imgs_test, imgs_id = mydata.load_test_data()
    imgs_train = np.reshape(imgs_train, (-1, img_rows, img_cols, 1))
    imgs_mask_train = np.reshape(imgs_mask_train, (-1, img_rows, img_cols, 1))
    imgs_test = np.reshape(imgs_test, (-1, img_rows, img_cols, 1))

    return imgs_train, imgs_mask_train, imgs_test, imgs_id

def train_and_predict():
    
    #parameter 
    batch_size = 20
    epochs = 50
    
    print('-'*30)
    print('Loading and preprocessing train data...')
    print('-'*30)
    imgs_train, imgs_mask_train, imgs_test, imgs_id = load_data()
    
    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization

    imgs_train -= mean
    imgs_train /= std

    imgs_mask_train = imgs_mask_train.astype('float32')
    imgs_mask_train /= 255.  # scale masks to [0, 1]
    
    x_train, y_train = imgs_train[:540], imgs_mask_train[:540]
    x_val, y_val = imgs_train[540:], imgs_mask_train[540:]
    
    datagen = ImageDataGenerator(contrast_stretching=False, adaptive_equalization=False, histogram_equalization=True, 
                                 featurewise_center=False, featurewise_std_normalization=False, 
                                 rotation_range=90.,width_shift_range=0.,height_shift_range=0.,shear_range=0.,zoom_range=0.,
                                 channel_shift_range=0.,
                                 horizontal_flip=True, vertical_flip=True)

    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_unet()
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    
    datagen.fit(x_train)
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=epochs, validation_data=(x_val, y_val), callbacks=[model_checkpoint])
    
    print('-'*30)
    print('Loading and preprocessing test data...')
    print('-'*30)

    imgs_test = imgs_test.astype('float32')
    imgs_test -= mean
    imgs_test /= std
    
    print('-'*30)
    print('Loading saved weights...')
    print('-'*30)
    model.load_weights('weights.h5')

    print('-'*30)
    print('Predicting masks on test data...')
    print('-'*30)
    
    imgs_mask_test = model.predict(imgs_test, verbose=1, batch_size= 5)
    np.save('imgs_mask_test.npy', imgs_mask_test)

    print('-' * 30)
    print('Saving predicted masks to files...')
    print('-' * 30)
    pred_dir = 'test/label'
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
    for image, image_id in zip(imgs_mask_test, imgs_id):
        image = (image[:, :, 0] * 255.).astype(np.uint8)
        imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)

In [3]:
if __name__ == '__main__':
    train_and_predict()

------------------------------
Loading and preprocessing train data...
------------------------------
------------------------------
Creating and compiling model...
------------------------------
------------------------------
Fitting model...
------------------------------
Epoch 1/50


C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\skimage\exposure\exposure.py:63: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel.
  warn("This might be a color image. The histogram will be "


27/27 [==============================] - 47s 2s/step - loss: -0.2175 - dice_coef: 0.2175 - val_loss: -0.2103 - val_dice_coef: 0.2103
Epoch 2/50
27/27 [==============================] - 38s 1s/step - loss: -0.2188 - dice_coef: 0.2188 - val_loss: -0.2117 - val_dice_coef: 0.2117
Epoch 3/50
27/27 [==============================] - 38s 1s/step - loss: -0.2188 - dice_coef: 0.2188 - val_loss: -0.2132 - val_dice_coef: 0.2132
Epoch 4/50
27/27 [==============================] - 38s 1s/step - loss: -0.2201 - dice_coef: 0.2201 - val_loss: -0.2147 - val_dice_coef: 0.2147
Epoch 5/50
27/27 [==============================] - 38s 1s/step - loss: -0.2209 - dice_coef: 0.2209 - val_loss: -0.2198 - val_dice_coef: 0.2198
Epoch 6/50
27/27 [==============================] - 38s 1s/step - loss: -0.2343 - dice_coef: 0.2343 - val_loss: -0.2360 - val_dice_coef: 0.2360
Epoch 7/50
27/27 [==============================] - 36s 1s/step - loss: -0.2454 - dice_coef: 0.2454 - val_loss: -0.2355 - val_dice_coef: 0.2355
Epo